In [9]:
import pandas as pd
import numpy as np

plays = pd.read_csv("./in/plays_with_passer_expanded_with_z_norm.csv")
name = lambda r : str(r['gameId']) + "-"+str(r['playId']) 
plays['name'] = plays.apply (lambda row: name(row), axis=1)
plays.head(5)

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,ball_snap_frame,play_direction,frames_to_pass,seconds_since_pass,los_norm,start_x_norm,end_x_norm,start_y_norm,end_y_norm,name
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,11,left,25,2.5,40,33.10,46.71,26.33,6.83,2018090600-75
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,11,left,20,2.0,40,34.39,43.69,23.62,16.91,2018090600-146
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,11,left,23,2.3,40,34.49,37.41,24.64,47.49,2018090600-168
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,11,left,28,2.8,40,30.60,61.83,26.24,41.47,2018090600-190
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,11,left,20,2.0,40,33.52,41.95,28.86,10.88,2018090600-256


In [5]:


filepath = "/Users/Rishav/Downloads/nfl-big-data-bowl-2021/"



week1 = pd.read_csv("/Users/Rishav/Downloads/nfl-big-data-bowl-2021/week1.csv")

# We also only care about the frame at which a catch happens, so lets remove all the other crap
week1 = week1[week1["event"] == "ball_snap"]


for i in range(2,18):
    print("Adding week %d" % i)
    current_week = pd.read_csv("/Users/Rishav/Downloads/nfl-big-data-bowl-2021/week%d_norm.csv" % i)

# We also only care about the frame at which a catch happens, so lets remove all the other crap
    current_week = current_week[current_week["event"] == "ball_snap"]
    week1 = week1.append(current_week, ignore_index = True)



week1.head(5)

Adding week 2
Adding week 3
Adding week 4
Adding week 5
Adding week 6
Adding week 7
Adding week 8
Adding week 9
Adding week 10
Adding week 11
Adding week 12
Adding week 13
Adding week 14
Adding week 15
Adding week 16
Adding week 17


,time,x,y,s,a,dis,o,dir,event,nflId,...,s_dir_rad,v_x,v_y,v_theta,v_mag,a_old,a_x,a_y,a_theta,a_mag
0,2018-09-07T01:07:15.599Z,91.74,26.67,0.00,0.00,0.01,277.41,235.01,ball_snap,310.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-09-07T01:07:15.599Z,88.89,36.50,0.01,0.01,0.00,104.49,36.45,ball_snap,79848.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-09-07T01:07:15.599Z,91.36,44.14,0.00,0.01,0.02,289.53,49.86,ball_snap,2495454.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-09-07T01:07:15.599Z,86.47,22.03,0.18,0.07,0.01,71.89,82.30,ball_snap,2495613.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-09-07T01:07:15.599Z,90.78,36.16,0.00,0.00,0.00,259.19,294.93,ball_snap,2533040.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
new_thing = []
for name in plays['name']:
    try:
        n = {}
        gid,pid = int(name.split("-")[0]), int(name.split("-")[1])
        curr_play = week1[(week1.gameId == gid) & (week1.playId == pid) ]
        sorted_by_line = curr_play.sort_values("y")
        ball_location = curr_play[curr_play.displayName == "Football"].iloc[0]['y']
        above_ball = sorted_by_line[sorted_by_line.y > ball_location]
        below_ball = sorted_by_line[sorted_by_line.y < ball_location]
        ab = list(reversed(list(above_ball.route.dropna())))
        bb = list(below_ball.route.dropna())

        above_name = "-".join(ab)
        below_name = "-".join(bb)
        print(above_name, below_name)
        n['Top_Routes'] = above_name
        n['Bottom_Routes'] = below_name
        n['gameId'] = gid
        n['playId'] = pid

        new_thing.append(n)
    except: pass

HITCH-HITCH OUT-FLAT
HITCH-FLAT HITCH-CROSS-FLAT
GO SLANT-SLANT-SCREEN-FLAT
 HITCH-IN-CORNER
GO-OUT GO
OUT-OUT-ANGLE OUT-HITCH
GO-OUT IN-CROSS-IN
HITCH HITCH-CROSS-FLAT
CROSS CROSS-CROSS-FLAT
CROSS-CROSS FLAT-CORNER-FLAT
IN-IN-FLAT HITCH
OUT-HITCH-HITCH IN-FLAT
IN-IN-FLAT GO-GO
POST POST-OUT
GO-FLAT GO-FLAT-HITCH
OUT CROSS-CROSS-CROSS-FLAT
HITCH-FLAT HITCH-OUT
SLANT-HITCH IN-IN-FLAT
GO-OUT-FLAT GO-IN
HITCH-HITCH GO-OUT-ANGLE
GO-ANGLE IN-CROSS-GO
SLANT-OUT POST-HITCH-FLAT
OUT-HITCH-FLAT 
CORNER-HITCH IN-HITCH-FLAT
GO-GO-FLAT IN
GO-GO-FLAT GO-HITCH
POST-WHEEL POST-CROSS-POST
CORNER-FLAT CORNER-FLAT
HITCH-IN-FLAT IN-IN
POST-OUT-FLAT SLANT-GO
IN-HITCH HITCH-OUT
POST-HITCH-CROSS HITCH-FLAT
HITCH-IN-CORNER IN-ANGLE
IN-POST GO-OUT-FLAT
HITCH-FLAT IN-IN-POST
GO-FLAT HITCH-HITCH-HITCH
CROSS GO-IN-CROSS-FLAT
IN SCREEN-GO-HITCH-FLAT
CROSS POST-FLAT
IN-GO-FLAT GO-GO
HITCH OUT-HITCH-HITCH
CROSS GO-CROSS-FLAT
GO-SLANT HITCH-GO-OUT
CROSS GO-GO-POST
SLANT-FLAT SLANT-HITCH-FLAT
HITCH-CROSS-WHEEL POST-C

HITCH-WHEEL POST-SLANT-ANGLE
GO-POST-FLAT GO-POST
GO-GO-OUT GO-OUT
HITCH-IN-FLAT OUT-CROSS
GO-OUT-CROSS GO-HITCH
OUT-CROSS HITCH-SLANT-FLAT
GO-ANGLE HITCH-OUT-OUT
HITCH-GO HITCH-GO-POST
IN-GO CROSS-OUT-ANGLE
HITCH-ANGLE HITCH-GO-FLAT
HITCH HITCH-SLANT-SLANT-SCREEN
POST HITCH-GO
CROSS-OUT GO-GO-CROSS
CORNER-FLAT IN-GO-CROSS
POST-OUT GO-IN-IN
HITCH-HITCH HITCH-OUT-GO
HITCH-SCREEN-ANGLE GO-IN
POST CROSS-CROSS
GO-OUT POST-SLANT
CROSS POST-CROSS-OUT
HITCH-GO-HITCH HITCH-GO
HITCH-POST HITCH-GO-CROSS
HITCH-SLANT HITCH-GO-OUT
OUT-ANGLE GO-CORNER-SCREEN
GO-OUT CORNER-HITCH
IN GO
CORNER-OUT POST-CROSS-ANGLE
HITCH-IN-FLAT CORNER-OUT
SCREEN FLAT-ANGLE
SLANT-SLANT-FLAT 
GO-IN HITCH-POST-OUT
SCREEN-FLAT-FLAT-FLAT GO
HITCH-IN GO-OUT
CORNER-FLAT HITCH-IN-GO
SLANT-OUT OUT-SLANT-ANGLE
SLANT-OUT-FLAT GO-OUT
GO-OUT-OUT IN-OUT
POST-IN-ANGLE GO-OUT
GO-GO-ANGLE GO-GO
HITCH-GO-ANGLE OUT-POST
GO-GO-FLAT GO-GO
HITCH-HITCH-FLAT GO-HITCH
GO-CROSS-SCREEN CROSS-FLAT
OUT-CROSS-POST GO-FLAT
GO-HITCH-GO GO-OUT
GO-CROS

GO-CORNER-FLAT IN-IN
GO-POST GO-IN-FLAT
SLANT-IN-CORNER OUT-OUT
IN-OUT-ANGLE HITCH-FLAT
GO-GO GO-POST
HITCH-FLAT HITCH-CROSS-FLAT
POST CROSS
SCREEN-GO HITCH-GO-SLANT
 HITCH-GO-CORNER
CORNER-SCREEN CROSS-CROSS
 GO-HITCH-FLAT
CROSS-CORNER-ANGLE IN-GO
CORNER-FLAT IN-OUT-OUT
HITCH GO-HITCH-OUT
SCREEN-FLAT-FLAT GO
GO-POST GO-HITCH-FLAT
CORNER-FLAT-FLAT CROSS
IN-HITCH GO-POST-FLAT
OUT-GO HITCH-IN
SLANT-SCREEN CROSS-GO-FLAT
GO-OUT-FLAT POST-CROSS
OUT-CROSS-FLAT IN-GO
HITCH-HITCH-FLAT HITCH-FLAT
GO-HITCH-FLAT GO-CORNER
POST-HITCH HITCH-GO
CROSS-IN POST-IN-FLAT
OUT-IN-POST HITCH-FLAT
CROSS-FLAT IN-CROSS-HITCH
HITCH-FLAT HITCH-FLAT-HITCH
IN-HITCH GO-CORNER-FLAT
POST-HITCH-CORNER IN-OUT
IN-POST-IN CORNER-ANGLE
SLANT-SLANT-GO CORNER
IN-SLANT-GO GO-OUT
SLANT-IN-CORNER SLANT-ANGLE
IN-HITCH GO-POST
IN-OUT GO-GO-FLAT
GO-GO GO-GO-ANGLE
HITCH-GO GO-IN-ANGLE
SLANT-OUT GO-GO-FLAT
FLAT FLAT-HITCH-HITCH
CORNER-FLAT POST-CROSS-CROSS
GO-SLANT HITCH-GO-OUT
SLANT-FLAT GO-OUT-GO
HITCH HITCH-GO-OUT
GO-POST-HITCH 

GO-OUT 
SLANT-IN-GO SLANT-FLAT
HITCH-GO-HITCH CORNER-ANGLE
HITCH-FLAT POST-OUT-CROSS
GO-FLAT-SCREEN SLANT-SLANT
GO-FLAT IN-HITCH-CROSS
GO-POST-ANGLE HITCH-GO
ANGLE-OUT HITCH-GO
GO-OUT GO-GO-CROSS
IN-ANGLE IN-GO-OUT
SCREEN-ANGLE GO-FLAT-FLAT
SCREEN-OUT SCREEN-OUT-OUT
FLAT-OUT SLANT-FLAT-ANGLE
HITCH-GO-HITCH OUT-ANGLE
CROSS GO-GO-GO-SCREEN
GO-HITCH GO-HITCH-HITCH
HITCH-FLAT HITCH-FLAT-ANGLE
GO-OUT GO-OUT-IN
GO-FLAT GO-HITCH-POST
HITCH-GO-HITCH POST-ANGLE
IN-ANGLE IN-GO-OUT
HITCH-ANGLE HITCH-GO-HITCH
IN-ANGLE IN-GO-OUT
HITCH-FLAT HITCH-GO-HITCH
POST-GO HITCH-GO-OUT
IN-OUT IN-POST-CORNER
HITCH-HITCH CORNER-FLAT
GO-OUT-FLAT HITCH-OUT
GO-SLANT GO-GO
POST-HITCH-FLAT IN-GO
HITCH-POST-FLAT IN-POST
GO IN-CROSS-FLAT
GO-GO-FLAT CROSS-CROSS
OUT-POST-CROSS CROSS
CORNER-GO-FLAT 
CROSS-ANGLE GO-FLAT
POST-CROSS-CROSS HITCH-FLAT
SLANT-IN GO-GO-OUT
GO POST-GO-FLAT
GO-CORNER-OUT POST
SLANT-FLAT GO-FLAT-FLAT
 CORNER-OUT-SCREEN
OUT POST-GO
IN-POST IN-OUT-ANGLE
GO-CROSS-SCREEN GO
GO-OUT GO-OUT-HITCH
GO-GO-HI

GO-OUT-OUT IN-ANGLE
CROSS-CROSS CORNER-FLAT
IN-CROSS-SLANT OUT
HITCH-FLAT HITCH-FLAT
GO-SCREEN-FLAT GO-GO
OUT-OUT POST-OUT-FLAT
SLANT SCREEN-SCREEN-SCREEN
SCREEN-GO GO-HITCH
GO OUT-OUT-FLAT
OUT-FLAT CROSS-CROSS
FLAT GO-SCREEN-FLAT
OUT IN-OUT-HITCH
POST-OUT GO-GO-FLAT
POST-OUT GO-IN-FLAT
 
CORNER-OUT GO-OUT-FLAT
IN POST
IN SCREEN-GO-OUT-FLAT
IN-FLAT HITCH-CORNER-CORNER
HITCH-HITCH-POST GO-FLAT
POST-CROSS CROSS-FLAT
SLANT-CROSS-CROSS HITCH-SCREEN
CROSS OUT-FLAT
GO-SCREEN SLANT-CROSS
GO-ANGLE HITCH-OUT-POST
 GO-GO-FLAT-FLAT
OUT-ANGLE OUT-HITCH-HITCH
GO-OUT SLANT-HITCH-HITCH
GO-HITCH GO-GO-GO
HITCH GO-POST-POST-ANGLE
SCREEN-GO-HITCH-SCREEN HITCH
GO GO-OUT
IN-OUT HITCH-FLAT-ANGLE
POST-POST FLAT-CORNER-ANGLE
HITCH-FLAT-ANGLE OUT-CORNER
HITCH-CROSS-FLAT CROSS
IN-FLAT HITCH-POST-FLAT
IN-GO-POST CROSS-FLAT
POST-POST GO-OUT-FLAT
IN-CROSS-SCREEN FLAT
POST-SLANT-OUT IN
GO-CROSS GO-FLAT-FLAT
POST-CROSS-FLAT GO-OUT
GO-OUT-FLAT POST-POST
GO-HITCH HITCH-CROSS-FLAT
POST-CROSS-IN GO-OUT
OUT-ANGLE IN-IN-

...

HITCH-FLAT CORNER-GO-FLAT
CORNER-HITCH-ANGLE HITCH-SCREEN
CROSS-FLAT GO-HITCH
SLANT-SCREEN SCREEN-FLAT-FLAT
IN-ANGLE POST-CORNER-ANGLE
GO SLANT-OUT
HITCH-GO GO-GO-OUT
OUT IN-GO
HITCH-IN-FLAT GO-OUT
POST-FLAT-OUT HITCH-GO
SLANT-GO-GO GO-OUT
ANGLE SCREEN-FLAT-SCREEN
IN-SLANT-CORNER CORNER
HITCH-SLANT-FLAT HITCH-FLAT
 SCREEN-GO-CROSS-ANGLE
POST-GO-HITCH CROSS-HITCH
GO-CROSS GO-FLAT-ANGLE
CROSS HITCH-OUT-HITCH-FLAT
FLAT IN-FLAT-ANGLE
OUT-OUT HITCH-FLAT-ANGLE
CROSS POST-CROSS
GO-OUT HITCH-IN-OUT
SLANT-HITCH HITCH-GO-HITCH
SLANT-HITCH-ANGLE OUT-HITCH
CORNER-FLAT-OUT CROSS-HITCH
GO-ANGLE GO
CROSS-CROSS-FLAT FLAT
OUT-FLAT POST
GO-OUT-CROSS GO
GO HITCH-CROSS-OUT
OUT-OUT-POST OUT-FLAT
GO-GO-GO GO-SCREEN
IN-CROSS-CROSS GO
HITCH-GO OUT-FLAT
HITCH-IN-GO POST
CROSS GO-CROSS-FLAT
HITCH SLANT-SCREEN-GO-FLAT
CORNER-CORNER-FLAT 
GO-OUT SCREEN-HITCH-IN
CROSS CORNER-HITCH-OUT
HITCH-POST HITCH-GO
GO-SCREEN GO-GO-CROSS
HITCH-HITCH HITCH-OUT-ANGLE
POST-CROSS OUT-FLAT-CROSS
CROSS-CROSS-CROSS CROSS-OUT
SLANT-F

GO-GO HITCH-POST-OUT
GO GO-POST-CORNER
OUT-CORNER-CROSS GO
GO-POST-GO GO-OUT
IN-CROSS POST-GO-SCREEN
SLANT-FLAT GO-FLAT-OUT
SCREEN CROSS-POST-CROSS
SLANT-SLANT-CROSS OUT-FLAT
OUT-HITCH-CROSS HITCH-ANGLE
CROSS-FLAT-FLAT POST
GO-POST-FLAT GO-FLAT
GO SLANT-OUT-ANGLE
HITCH-FLAT-FLAT HITCH-CROSS
POST-IN-POST HITCH-GO
IN-CROSS-IN IN-FLAT
HITCH-CORNER-FLAT-FLAT CROSS
HITCH-ANGLE HITCH-GO-OUT
HITCH-ANGLE HITCH-HITCH-GO
OUT-HITCH-ANGLE OUT-HITCH
HITCH-HITCH HITCH-ANGLE
SCREEN-GO-OUT SLANT-ANGLE
 GO-SCREEN-CROSS-FLAT
HITCH-GO SLANT-HITCH-FLAT
HITCH-CORNER HITCH-CORNER-FLAT
CROSS-FLAT IN
GO-ANGLE SLANT-GO-FLAT
CORNER-CROSS-FLAT 
GO-POST-GO GO-FLAT
HITCH-ANGLE SLANT-CROSS-GO
IN-FLAT HITCH-FLAT-HITCH
HITCH-FLAT IN-HITCH-GO
SLANT-FLAT HITCH-GO-HITCH
GO-HITCH-HITCH HITCH-FLAT
HITCH-IN GO-POST-FLAT
SLANT-HITCH-HITCH CORNER-FLAT
GO IN-OUT-GO
HITCH HITCH-SLANT
GO-CROSS GO-GO
HITCH-SLANT-CORNER OUT
GO-FLAT GO-GO-SCREEN
GO-CROSS-IN GO
GO-HITCH GO-HITCH-FLAT
HITCH-HITCH-FLAT OUT
HITCH-HITCH-FLAT POST-HITCH

SLANT-FLAT-SLANT GO-OUT
GO-FLAT GO-GO-CROSS
GO GO-OUT-IN
HITCH-CROSS HITCH
GO-POST-FLAT HITCH-OUT
SLANT-FLAT-ANGLE SCREEN
SLANT-FLAT-HITCH GO-OUT
SLANT-CROSS GO-OUT-FLAT
GO-GO-POST GO-ANGLE
CROSS-CROSS-FLAT POST-ANGLE
GO-OUT-OUT 
 GO-CROSS-GO-SCREEN
SLANT-GO-GO HITCH
HITCH-FLAT SLANT-SLANT-FLAT
GO-OUT POST-CROSS-CROSS
HITCH-FLAT GO-FLAT-OUT
HITCH HITCH-CROSS
GO GO-CROSS-FLAT
GO IN-GO-POST
CROSS-CROSS OUT-CROSS-FLAT
HITCH-CORNER POST-CORNER-ANGLE
OUT-ANGLE GO-OUT-HITCH
OUT-ANGLE IN-HITCH-HITCH
SLANT-FLAT SCREEN-HITCH-IN
SCREEN HITCH-GO-WHEEL
HITCH-HITCH-FLAT HITCH-FLAT
SCREEN GO-HITCH-CORNER
HITCH-POST GO-GO-ANGLE
GO-SLANT-SCREEN HITCH-HITCH
IN-HITCH HITCH-FLAT
CROSS HITCH
GO-FLAT-OUT IN-FLAT
OUT-HITCH SLANT-FLAT
CROSS-FLAT SCREEN-OUT-HITCH
CROSS POST-CROSS-WHEEL
POST-ANGLE CROSS-CROSS
CORNER-FLAT GO-GO-ANGLE
 IN-POST-FLAT
CROSS-HITCH-WHEEL CROSS-CORNER
SLANT-CORNER GO
CROSS-FLAT GO-GO
OUT-FLAT CROSS-SCREEN
HITCH-CORNER-FLAT 
GO-GO-HITCH CROSS-FLAT
 
 
HITCH-WHEEL HITCH-CROSS-FLAT
HITCH

GO-IN-FLAT OUT-CROSS
IN-IN-GO-FLAT HITCH
HITCH-ANGLE FLAT
SCREEN-SCREEN-ANGLE 
CROSS-ANGLE OUT-ANGLE
HITCH-FLAT POST-GO-HITCH
CORNER-CROSS-OUT POST
HITCH-IN-FLAT GO-OUT
HITCH-FLAT IN-HITCH-FLAT
GO-POST GO-POST-ANGLE
HITCH HITCH-GO-FLAT
POST-POST GO-CROSS-ANGLE
CORNER-FLAT OUT-FLAT-ANGLE
OUT-POST OUT-HITCH-FLAT
GO-OUT CROSS-HITCH-FLAT
OUT-FLAT IN-IN-CROSS
HITCH-CROSS GO-HITCH
HITCH-FLAT-HITCH HITCH-FLAT
CROSS-GO IN-CROSS-HITCH
HITCH-FLAT HITCH-SLANT-ANGLE
IN CORNER-POST-FLAT-FLAT
IN-FLAT HITCH-HITCH-HITCH
SLANT-POST-OUT IN
POST-HITCH CROSS-IN
GO-ANGLE IN-GO-HITCH
GO-CROSS CORNER-SCREEN
GO-CROSS SLANT-SCREEN-FLAT
CROSS-CROSS HITCH-FLAT-FLAT
GO-SLANT-IN CROSS-FLAT
GO-OUT GO
GO-POST-FLAT POST-SLANT
SLANT-POST SLANT-OUT-HITCH
GO-POST GO-IN-GO
IN-OUT SLANT-GO-OUT
IN HITCH-OUT-FLAT-FLAT
GO-CROSS GO-FLAT
POST-HITCH-FLAT HITCH-HITCH
CROSS-IN CROSS-HITCH-WHEEL
IN-SCREEN CROSS-SCREEN-GO
HITCH-HITCH-HITCH HITCH-OUT
IN-WHEEL IN-IN-CROSS
SLANT GO-SLANT-FLAT-SCREEN
CROSS-FLAT OUT-IN-HITCH
IN CORNER-O

CROSS GO-SCREEN
GO-SCREEN HITCH-OUT-OUT
CROSS-IN-CROSS-FLAT OUT
GO-CROSS-SCREEN CROSS-GO
HITCH-HITCH-POST IN-FLAT
HITCH HITCH-CORNER-FLAT
SLANT-FLAT SLANT-OUT-HITCH
CROSS-FLAT-CROSS GO
HITCH-CROSS-SCREEN HITCH-FLAT
CROSS-FLAT CROSS
CROSS-CROSS GO-CROSS-GO
CORNER-CORNER-CROSS-FLAT POST
GO-IN HITCH-POST-HITCH
SLANT-POST-OUT IN-ANGLE
SLANT-OUT-HITCH SLANT-FLAT
GO-CROSS-IN OUT-SCREEN
IN-FLAT SLANT-HITCH-FLAT
CROSS-CROSS-CROSS GO
IN-CROSS-CORNER HITCH-FLAT
GO-POST-POST GO-SCREEN
GO IN-POST-FLAT
HITCH-HITCH GO-OUT-FLAT
HITCH-IN-GO OUT-FLAT
GO-POST-HITCH SLANT
GO-CROSS CROSS-IN
HITCH-GO CROSS-IN
CORNER-GO-OUT 
GO-GO-ANGLE GO-GO
CROSS-SCREEN GO-CORNER-FLAT
GO HITCH-HITCH-FLAT
GO-GO-ANGLE GO-GO
CROSS-FLAT-FLAT GO-CROSS
CROSS-FLAT GO-CROSS-GO
GO-HITCH GO-HITCH
CROSS-WHEEL GO-CROSS-HITCH
GO-GO-CROSS OUT-WHEEL
HITCH-GO-HITCH HITCH
CROSS-IN GO-HITCH
GO-POST CROSS-CROSS-FLAT
GO-IN-HITCH CROSS-FLAT
OUT-ANGLE CROSS-CORNER-POST
OUT-CROSS IN-POST
GO-OUT POST-CROSS-CROSS
HITCH-GO CORNER-CROSS-OUT
GO-CROS

In [42]:
nt = pd.DataFrame(new_thing)
display(nt)

,Top_Routes,Bottom_Routes,gameId,playId
0,HITCH-HITCH,OUT-FLAT,2018090600,75
1,HITCH-FLAT,HITCH-CROSS-FLAT,2018090600,146
2,GO,SLANT-SLANT-SCREEN-FLAT,2018090600,168
3,,HITCH-IN-CORNER,2018090600,190
4,GO-OUT,GO,2018090600,256
...,...,...,...,...
17340,GO-ANGLE,SLANT-GO-CORNER,2018122201,3842
17341,IN-CORNER,CROSS-OUT,2018122201,3884
17342,HITCH-GO,HITCH-POST,2018122201,3909
17343,GO-CROSS,POST-HITCH-ANGLE,2018122201,3931


In [44]:
p = plays.merge(nt)
display(p)

p.to_csv("plays_with_route_combos.csv")

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,playType,yardlineSide,yardlineNumber,...,frames_to_pass,seconds_since_pass,los_norm,start_x_norm,end_x_norm,start_y_norm,end_y_norm,name,Top_Routes,Bottom_Routes
0,2018090600,75,(15:00) M.Ryan pass short right to J.Jones pus...,1,1,15,ATL,play_type_pass,ATL,20,...,25,2.5,40,33.10,46.71,26.33,6.83,2018090600-75,HITCH-HITCH,OUT-FLAT
1,2018090600,146,(13:10) M.Ryan pass incomplete short right to ...,1,1,10,ATL,play_type_pass,PHI,39,...,20,2.0,40,34.39,43.69,23.62,16.91,2018090600-146,HITCH-FLAT,HITCH-CROSS-FLAT
2,2018090600,168,(13:05) (Shotgun) M.Ryan pass incomplete short...,1,2,10,ATL,play_type_pass,PHI,39,...,23,2.3,40,34.49,37.41,24.64,47.49,2018090600-168,GO,SLANT-SLANT-SCREEN-FLAT
3,2018090600,190,(13:01) (Shotgun) M.Ryan pass deep left to J.J...,1,3,10,ATL,play_type_pass,PHI,39,...,28,2.8,40,30.60,61.83,26.24,41.47,2018090600-190,,HITCH-IN-CORNER
4,2018090600,256,(10:59) (Shotgun) M.Ryan pass incomplete short...,1,3,1,ATL,play_type_pass,PHI,1,...,20,2.0,40,33.52,41.95,28.86,10.88,2018090600-256,GO-OUT,GO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17340,2018122201,3842,(2:19) (Shotgun) P.Rivers pass short right to ...,4,2,2,LAC,play_type_pass,LAC,33,...,18,1.8,40,32.10,43.08,29.32,18.22,2018122201-3842,GO-ANGLE,SLANT-GO-CORNER
17341,2018122201,3884,(2:00) (Shotgun) P.Rivers pass short left to K...,4,1,10,LAC,play_type_pass,LAC,40,...,28,2.8,40,31.32,54.28,24.20,35.81,2018122201-3884,IN-CORNER,CROSS-OUT
17342,2018122201,3909,(1:41) (Shotgun) P.Rivers pass incomplete shor...,4,1,10,LAC,play_type_pass,BAL,43,...,24,2.4,40,32.94,52.68,24.28,2.40,2018122201-3909,HITCH-GO,HITCH-POST
17343,2018122201,3931,(1:37) (Shotgun) P.Rivers pass incomplete deep...,4,2,10,LAC,play_type_pass,BAL,43,...,25,2.5,40,31.27,65.72,26.15,45.57,2018122201-3931,GO-CROSS,POST-HITCH-ANGLE


In [50]:
counts = p.value_counts(["Top_Routes"])
counts2 = p.value_counts(["Bottom_Routes"])
display(counts[:10])


Top_Routes
GO            688
HITCH-FLAT    516
GO-OUT        443
GO-FLAT       443
CROSS         414
              356
SLANT-FLAT    330
HITCH         321
CROSS-FLAT    248
GO-ANGLE      247
dtype: int64

In [51]:
display(counts2[:10])

Bottom_Routes
GO               603
HITCH-FLAT       515
                 416
GO-OUT           401
GO-FLAT          382
HITCH            310
SLANT-FLAT       279
GO-GO            269
CROSS            267
OUT              246
dtype: int64

In [53]:
counts3 = p.value_counts(["Top_Routes", "Bottom_Routes"])
display(counts3)

Top_Routes                               Bottom_Routes   
                                                             164
HITCH-HITCH-FLAT                         HITCH-FLAT           49
HITCH-FLAT                               HITCH-HITCH-FLAT     46
                                         HITCH-FLAT-HITCH     29
GO-GO                                    GO-GO                27
                                                            ... 
HITCH-HITCH-SLANT                        GO-ANGLE              1
                                         HITCH-FLAT            1
                                         HITCH-GO              1
                                         HITCH-HITCH           1
undefined-undefined-undefined-undefined  undefined             1
Length: 11712, dtype: int64